In [ ]:
# ============================================================================
# CELL 0: Suppress ALL TensorFlow/CUDA Warnings (RUN THIS FIRST!)
# ============================================================================

import os
import sys
import warnings

# CRITICAL: Set these BEFORE importing TensorFlow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress TF logging
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'  # Suppress oneDNN messages
os.environ['TF_CPP_MIN_VLOG_LEVEL'] = '3'  # Suppress verbose logging
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # Use only first GPU (optional)

# Suppress Python warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# Redirect stderr temporarily to suppress XLA messages
import io
import contextlib

# Suppress all stderr output during TensorFlow import
stderr_backup = sys.stderr
sys.stderr = io.StringIO()

# Now import TensorFlow
import tensorflow as tf

# Restore stderr
sys.stderr = stderr_backup

# Additional TensorFlow configuration
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(0)

# Suppress absl logging
import logging
logging.getLogger('absl').setLevel(logging.ERROR)
logging.getLogger('tensorflow').setLevel(logging.ERROR)

# Configure GPU memory growth (prevents memory warnings)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ {len(gpus)} GPU(s) configured with memory growth")
    except RuntimeError as e:
        print(f"GPU configuration: {e}")
else:
    print("✅ Running on CPU")

print("✅ All warnings suppressed successfully\n")

In [ ]:
# ============================================================================
# CELL 1: Import Libraries and Suppress Warnings
# ============================================================================

import os
import warnings

# Suppress TensorFlow warnings FIRST
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
warnings.filterwarnings('ignore')

# Core imports
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.metrics import (
    roc_curve, auc, precision_recall_curve, average_precision_score,
    confusion_matrix, classification_report, f1_score,
    accuracy_score, precision_score, recall_score,
    roc_auc_score
)
from sklearn.preprocessing import label_binarize
from scipy import stats
import itertools
from sklearn.calibration import calibration_curve
from tqdm import tqdm
import math
from scipy.spatial.distance import pdist, squareform
from itertools import combinations
from sklearn.metrics.pairwise import cosine_similarity as sklearn_cosine_similarity
import pickle
import json

# TensorFlow imports
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
import logging

# Configure TensorFlow logging
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
logging.getLogger('tensorflow').setLevel(logging.ERROR)
logging.getLogger('absl').setLevel(logging.ERROR)

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("✅ All libraries imported successfully")
print("✅ Warnings suppressed\n")


In [ ]:
# Global config
IMG_SIZE = 112
BATCH_SIZE = 64  # Increased for better gradient estimates
AUTOTUNE = tf.data.AUTOTUNE
SEED = 42

# Set random seeds for reproducibility
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Dataset path - UPDATE THIS TO YOUR PATH
DATASET_DIR = "/kaggle/input/datasetsforrestnet/ThirdLap"

print("✅ Configuration loaded")
print(f"Image Size: {IMG_SIZE}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Dataset Directory: {DATASET_DIR}\n")


In [ ]:
# ============================================================================
# CELL 3: Data Loading Functions
# ============================================================================

def parse_image(file_path, label):
    """Parse and preprocess image"""
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label


def build_train_val_datasets_class_disjoint(root_dir, val_identities=100):
    """
    Build datasets with class-disjoint split.
    Training and validation use COMPLETELY DIFFERENT identities.
    
    This is the MOST REALISTIC evaluation:
    - Training: Person A, B, C (all images)
    - Validation: Person D, E, F (all images) ← Never seen during training!
    - Tests TRUE generalization to new identities
    
    Args:
        root_dir: Root directory containing identity folders
        val_identities: Number of identities to reserve for validation
    
    Returns:
        train_ds, val_ds, num_train_classes
    """
    class_names = sorted(
        [d for d in os.listdir(root_dir)
         if os.path.isdir(os.path.join(root_dir, d))]
    )

    total_identities = len(class_names)
    print(f"Total identities in dataset: {total_identities}")
    
    # Shuffle to randomly assign identities to train/val
    random.shuffle(class_names)
    
    # Split identities (not images!)
    val_identities = min(val_identities, total_identities // 5)  # At most 20% for val
    val_classes = class_names[:val_identities]
    train_classes = class_names[val_identities:]
    
    print(f"\n{'='*70}")
    print("CLASS-DISJOINT SPLIT (Most Realistic Evaluation)")
    print('='*70)
    print(f"Training identities: {len(train_classes)}")
    print(f"Validation identities: {len(val_classes)}")
    print(f"Split: {len(train_classes)}/{len(val_classes)} (train/val)")
    print("\n⚠️  NOTE: Validation accuracy will be 0% (this is expected!)")
    print("   The model is trained on different identities than validation.")
    print("   ROC-AUC is the correct metric to monitor.\n")

    # Collect training data (use ALL images from training identities)
    train_paths, train_labels = [], []
    for idx, cls in enumerate(train_classes):
        cls_dir = os.path.join(root_dir, cls)
        images = [
            os.path.join(cls_dir, img)
            for img in os.listdir(cls_dir)
            if img.lower().endswith((".jpg", ".jpeg", ".png"))
        ]
        
        train_paths.extend(images)
        train_labels.extend([idx] * len(images))
    
    # Collect validation data (use ALL images from validation identities)
    val_paths, val_labels = [], []
    for idx, cls in enumerate(val_classes):
        cls_dir = os.path.join(root_dir, cls)
        images = [
            os.path.join(cls_dir, img)
            for img in os.listdir(cls_dir)
            if img.lower().endswith((".jpg", ".jpeg", ".png"))
        ]
        
        val_paths.extend(images)
        val_labels.extend([idx] * len(images))

    # ========================================================================
    # DATA LEAKAGE CHECK - Verify no image appears in both train and val
    # ========================================================================
    train_paths_set = set(train_paths)
    val_paths_set = set(val_paths)
    overlap = train_paths_set.intersection(val_paths_set)

    print(f"{'='*70}")
    print("DATA LEAKAGE VERIFICATION")
    print('='*70)
    print(f"Training images: {len(train_paths_set):,}")
    print(f"Validation images: {len(val_paths_set):,}")
    print(f"Overlapping images: {len(overlap):,}")

    if len(overlap) > 0:
        print("\n🚨 CRITICAL ERROR: DATA LEAKAGE DETECTED!")
        print(f"   {len(overlap)} images appear in BOTH train and validation!")
        print("   YOUR RESULTS ARE INVALID!")
        print("\n   First 5 overlapping files:")
        for idx, path in enumerate(list(overlap)[:5]):
            print(f"   {idx+1}. {path}")
        raise ValueError("Data leakage detected! Fix dataset split.")
    else:
        print("\n✅ PASS: No data leakage detected")
        print("   Train and validation sets are properly separated")

    # Verify label overlap (should NOT overlap for class-disjoint)
    train_labels_set = set(train_labels)
    val_labels_set = set(val_labels)
    label_overlap = train_labels_set.intersection(val_labels_set)

    print(f"\nLabel overlap: {len(label_overlap)} classes")
    if len(label_overlap) == 0:
        print("✅ CORRECT: Class-disjoint split confirmed")
        print("   Training and validation use completely different identities")
    else:
        print("⚠️  WARNING: Labels overlap detected!")
        print(f"   {len(label_overlap)} classes appear in both sets")
    print('='*70 + "\n")

    # Create datasets
    train_ds = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
    train_ds = train_ds.shuffle(20000, seed=SEED)
    train_ds = train_ds.map(parse_image, num_parallel_calls=AUTOTUNE)
    train_ds = train_ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)

    val_ds = tf.data.Dataset.from_tensor_slices((val_paths, val_labels))
    val_ds = val_ds.map(parse_image, num_parallel_calls=AUTOTUNE)
    val_ds = val_ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)

    num_train_classes = len(train_classes)
    
    return train_ds, val_ds, num_train_classes


print("✅ Data loading functions defined\n")


In [ ]:
 #============================================================================
# CELL 4: Load Dataset
# ============================================================================

print("Loading datasets with CLASS-DISJOINT split...")
print("This measures TRUE generalization to unseen identities!\n")

train_ds, val_ds, num_classes = build_train_val_datasets_class_disjoint(
    root_dir=DATASET_DIR,
    val_identities=100  # Reserve 100 identities for validation
)

print(f"\n✅ Datasets loaded successfully")
print(f"Model will be trained on {num_classes} identities")
print(f"Validation tests on completely unseen identities\n")



In [ ]:
# ============================================================================
# CELL 5: Model Architecture
# ============================================================================

def l2_norm(x):
    """L2 normalization layer"""
    return tf.nn.l2_normalize(x, axis=1)


def build_resnet_embedding():
    """Build ResNet50 backbone for face embeddings"""
    inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3), name="input_image")
    
    # Load ResNet50 with ImageNet weights
    base = ResNet50(
        include_top=False,
        weights="imagenet",
        input_tensor=inputs,
        pooling="avg"
    )
    
    # Fine-tuning strategy: freeze early layers
    base.trainable = True
    for layer in base.layers[:100]:
        layer.trainable = False
    
    # Embedding layers
    x = BatchNormalization(name='bn_before_dense')(base.output)
    x = Dense(512, use_bias=False, name='dense_embedding')(x)
    x = BatchNormalization(name='bn_after_dense')(x)
    embeddings = Lambda(l2_norm, name="embeddings")(x)
    
    return Model(inputs, embeddings, name="ResNet50_Embedding")


class ArcFace(tf.keras.layers.Layer):
    """
    ArcFace layer for face recognition.
    Adds angular margin to improve discriminative power.
    
    Reference: ArcFace: Additive Angular Margin Loss for Deep Face Recognition
    """
    def __init__(self, num_classes, margin=0.5, scale=64, **kwargs):
        super(ArcFace, self).__init__(**kwargs)
        self.num_classes = num_classes
        self.margin = margin  # Angular margin
        self.scale = scale    # Feature scale
        self.cos_m = tf.math.cos(margin)
        self.sin_m = tf.math.sin(margin)
        self.threshold = tf.math.cos(math.pi - margin)
        
    def build(self, input_shape):
        self.W = self.add_weight(
            name="W",
            shape=(input_shape[-1], self.num_classes),
            initializer=tf.keras.initializers.glorot_uniform(),
            regularizer=tf.keras.regularizers.l2(5e-4),
            trainable=True
        )
        super().build(input_shape)
        
    def call(self, inputs, labels=None, training=None):
        # Normalize weights and inputs
        W_norm = tf.nn.l2_normalize(self.W, axis=0)
        x_norm = tf.nn.l2_normalize(inputs, axis=1)
        
        # Compute cosine similarity
        cosine = tf.matmul(x_norm, W_norm)
        
        if labels is not None and training:
            # One-hot encode labels
            one_hot_labels = tf.one_hot(labels, depth=self.num_classes)
            
            # Compute theta and add angular margin
            theta = tf.acos(tf.clip_by_value(cosine, -1.0 + 1e-7, 1.0 - 1e-7))
            target_logit = tf.cos(theta + self.margin)
            
            # Combine target and non-target logits
            logits = cosine * (1 - one_hot_labels) + target_logit * one_hot_labels
            logits = logits * self.scale
        else:
            logits = cosine * self.scale
        
        return logits
    
    def get_config(self):
        config = super().get_config()
        config.update({
            'num_classes': self.num_classes,
            'margin': self.margin,
            'scale': self.scale
        })
        return config


print("✅ Model architecture defined\n")



In [ ]:
# ============================================================================
# CELL 6: Build and Compile Model
# ============================================================================

print("Building face recognition model...")

# Build backbone
backbone = build_resnet_embedding()
print("\nBackbone architecture:")
backbone.summary()

# Build ArcFace layer
arcface = ArcFace(num_classes, margin=0.5, scale=64)

# Create training model
inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3), name="input_image")
labels = Input(shape=(), name="label", dtype=tf.int32)

embeddings = backbone(inputs)
logits = arcface(embeddings, labels, training=True)

# Training model
train_model = Model([inputs, labels], logits, name="ArcFace_Trainer")

# Compile with fixed learning rate (will be adjusted by ReduceLROnPlateau)
train_model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

train_model.build([(None, IMG_SIZE, IMG_SIZE, 3), (None,)])

print(f"\n✅ Training model compiled")
print(f"Total parameters: {train_model.count_params():,}")
print(f"Trainable parameters: {sum([tf.size(w).numpy() for w in train_model.trainable_weights]):,}")

# Create inference model (for extracting embeddings)
inference_model = Model(inputs=backbone.input, outputs=backbone.output, 
                       name="Face_Embedding_Model")

print("✅ Inference model created for embedding extraction\n")


In [ ]:
# ============================================================================
# CELL 7: Prepare Datasets for Training
# ============================================================================

print("Preparing datasets for training...")

def prepare_train_dataset(dataset):
    """Convert dataset to format expected by training model"""
    def map_fn(images, labels):
        return (images, labels), labels
    return dataset.map(map_fn, num_parallel_calls=AUTOTUNE)

# Prepare datasets
train_ds_for_model = prepare_train_dataset(train_ds)
val_ds_for_model = prepare_train_dataset(val_ds)

print("✅ Datasets prepared for ArcFace training\n")

In [ ]:
# ============================================================================
# CELL 8: Training Callbacks
# ============================================================================

print("Setting up callbacks...")

# Helper function for ROC-AUC computation
def compute_similarity_scores_for_auc(embeddings, labels):
    """Compute similarity scores for ROC-AUC calculation"""
    n = len(embeddings)
    
    # Sample for efficiency
    if n > 2000:
        indices = np.random.choice(n, 2000, replace=False)
        embeddings = embeddings[indices]
        labels = labels[indices]
        n = 2000
    
    # Compute cosine similarity
    similarity_matrix = sklearn_cosine_similarity(embeddings)
    
    # Sample pairs
    scores = []
    pair_labels = []
    num_pairs = min(50000, n * (n - 1) // 2)
    sampled = 0
    
    while sampled < num_pairs:
        i, j = np.random.randint(0, n, 2)
        if i == j:
            continue
        scores.append(similarity_matrix[i, j])
        pair_labels.append(1 if labels[i] == labels[j] else 0)
        sampled += 1
    
    return np.array(scores), np.array(pair_labels)


# Custom callback for ROC-AUC monitoring
class ValidationROCAUCCallback(tf.keras.callbacks.Callback):
    """Monitor ROC-AUC on validation set - THE KEY METRIC for class-disjoint validation"""
    def __init__(self, backbone, val_ds):
        super().__init__()
        self.backbone = backbone
        self.val_ds = val_ds
        self.best_auc = 0.0
        self.auc_history = []
        
        # Extract validation labels once
        print("Extracting validation labels...")
        self.val_labels = []
        for _, labels in tqdm(val_ds, desc="Getting labels"):
            self.val_labels.append(labels.numpy())
        self.val_labels = np.concatenate(self.val_labels)
        print(f"Validation set: {len(self.val_labels)} samples\n")
    
    def on_epoch_end(self, epoch, logs=None):
        print(f"\n{'='*70}")
        print(f"Epoch {epoch+1}: Computing ROC-AUC on validation set...")
        print('='*70)
        
        # Extract embeddings
        embeddings_list = []
        for images, _ in tqdm(self.val_ds, desc="Extracting embeddings", leave=False):
            emb = self.backbone(images, training=False)
            embeddings_list.append(emb.numpy())
        embeddings = np.vstack(embeddings_list)
        
        # Compute ROC-AUC
        scores, pair_labels = compute_similarity_scores_for_auc(
            embeddings, self.val_labels
        )
        
        if len(np.unique(pair_labels)) > 1:
            roc_auc = roc_auc_score(pair_labels, scores)
            self.auc_history.append(roc_auc)
            logs['val_roc_auc'] = roc_auc
            
            print(f"ROC-AUC: {roc_auc:.4f}", end="")
            
            if roc_auc > self.best_auc:
                self.best_auc = roc_auc
                print(f" ✅ NEW BEST!")
            else:
                print(f" (best: {self.best_auc:.4f})")
        else:
            print("⚠️  Warning: Not enough positive/negative pairs")
        
        print('='*70)


# Setup callbacks
callbacks = [
    # ROC-AUC monitoring (MOST IMPORTANT for class-disjoint validation)
    ValidationROCAUCCallback(backbone, val_ds),
    
    # Model checkpoint - save best based on ROC-AUC
    ModelCheckpoint(
        'best_arcface_model.keras',
        monitor='val_roc_auc',
        mode='max',
        save_best_only=True,
        verbose=1
    ),
    
    # Learning rate reduction
    ReduceLROnPlateau(
        monitor='val_roc_auc',
        mode='max',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    ),
    
    # Early stopping
    EarlyStopping(
        monitor='val_roc_auc',
        mode='max',
        patience=8,
        restore_best_weights=True,
        verbose=1
    ),
    
    # TensorBoard
    tf.keras.callbacks.TensorBoard(
        log_dir='./logs',
        histogram_freq=1
    )
]


print("✅ Callbacks configured")
print("🔑 KEY: Monitoring 'val_roc_auc' (ignore val_accuracy, it will be 0%)\n")


In [ ]:
# ============================================================================
# CELL 9: Train the Model
# ============================================================================

print("="*70)
print("STARTING TRAINING")
print("="*70)
print("\n⚠️  REMINDER: val_accuracy will be 0% (this is NORMAL)")
print("   We're testing on completely unseen identities.")
print("   ROC-AUC is the metric that matters!\n")

EPOCHS = 40

history = train_model.fit(
    train_ds_for_model,
    validation_data=val_ds_for_model,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1  # Cleaner output: one line per epoch
)

print("\n" + "="*70)
print("✅ TRAINING COMPLETED")
print("="*70)
if callbacks[0].auc_history:
    print(f"Best ROC-AUC: {max(callbacks[0].auc_history):.4f}")
    print(f"Final ROC-AUC: {callbacks[0].auc_history[-1]:.4f}")
print("="*70 + "\n")



In [ ]:
# ============================================================================
# CELL 10: Save Model and History
# ============================================================================

print("Saving model and training history...")

# Save training history
history_dict = {}
for key, values in history.history.items():
    if isinstance(values, list):
        history_dict[key] = [float(v) for v in values]

with open('training_history.json', 'w') as f:
    json.dump(history_dict, f, indent=4)

with open('training_history.pkl', 'wb') as f:
    pickle.dump(history.history, f)

# Save complete model
train_model.save("arcface_resnet50_final.keras")

# Save only the backbone for inference
inference_model.save("embedding_model.keras")

print("✅ Models and history saved")
print("Files created:")
print("  - best_arcface_model.keras (best checkpoint)")
print("  - arcface_resnet50_final.keras (final model)")
print("  - embedding_model.keras (inference only)")
print("  - training_history.json")
print("  - training_history.pkl\n")

In [ ]:
# ============================================================================
# CELL 11: Evaluation Functions
# ============================================================================

def extract_embeddings(model, dataset):
    """Extract embeddings from dataset"""
    embeddings_list = []
    labels_list = []
    
    print("Extracting embeddings...")
    for images, labels in tqdm(dataset, desc="Processing batches"):
        embeddings = model.predict(images, verbose=0)
        embeddings_list.append(embeddings)
        labels_list.append(labels.numpy())
    
    return np.vstack(embeddings_list), np.concatenate(labels_list)


def compute_similarity_scores(embeddings, labels):
    """Compute similarity scores for verification"""
    n = len(embeddings)
    
    # Sample for efficiency if too many
    if n > 2000:
        indices = np.random.choice(n, 2000, replace=False)
        embeddings = embeddings[indices]
        labels = labels[indices]
        n = 2000
    
    similarity_matrix = sklearn_cosine_similarity(embeddings)
    
    # Create pair labels
    pair_labels = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            pair_labels[i, j] = 1 if labels[i] == labels[j] else 0
    
    # Get upper triangle (excluding diagonal)
    mask = np.triu_indices(n, k=1)
    scores = similarity_matrix[mask]
    labels_flat = pair_labels[mask]
    
    return scores, labels_flat


print("✅ Evaluation functions defined\n")


In [ ]:
# ============================================================================
# CELL 12: Extract Embeddings and Compute Metrics
# ============================================================================

print("="*70)
print("EVALUATION: Extracting embeddings and computing metrics")
print("="*70)

# Extract validation embeddings
print("\nExtracting validation embeddings...")
val_embeddings, val_labels = extract_embeddings(inference_model, val_ds)

print(f"\nValidation set:")
print(f"  Embeddings: {val_embeddings.shape}")
print(f"  Unique identities: {len(np.unique(val_labels))}")

# Compute similarity scores
print("\nComputing similarity scores...")
scores, pair_labels = compute_similarity_scores(val_embeddings, val_labels)

# Compute ROC curve
print("Computing ROC curve...")
fpr, tpr, thresholds = roc_curve(pair_labels, scores)
roc_auc = auc(fpr, tpr)

# Compute Precision-Recall curve
print("Computing Precision-Recall curve...")
precision, recall, pr_thresholds = precision_recall_curve(pair_labels, scores)
avg_precision = average_precision_score(pair_labels, scores)

# Find optimal threshold
print("Finding optimal threshold...")
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-10)
best_f1_idx = np.argmax(f1_scores)
best_threshold = pr_thresholds[best_f1_idx] if best_f1_idx < len(pr_thresholds) else 0.5
best_f1 = f1_scores[best_f1_idx]

# Compute TAR at specific FARs
tar_at_far = {}
for far_target in [0.001, 0.0001]:
    idx = np.argmin(np.abs(fpr - far_target))
    tar_at_far[far_target] = tpr[idx]
    threshold_at_far = thresholds[idx]

print("\n" + "="*70)
print("EVALUATION RESULTS")
print("="*70)
print(f"ROC-AUC: {roc_auc:.4f}")
print(f"Average Precision: {avg_precision:.4f}")
print(f"Best F1 Score: {best_f1:.4f} (threshold: {best_threshold:.4f})")
print(f"\nTrue Acceptance Rate (TAR) at Fixed False Acceptance Rate (FAR):")
print(f"  TAR @ FAR=0.1%:  {tar_at_far[0.001]:.4f}")
print(f"  TAR @ FAR=0.01%: {tar_at_far[0.0001]:.4f}")
print("="*70 + "\n")


In [ ]:
# ============================================================================
# CELL 13: Plot Training History
# ============================================================================

print("Plotting training history...")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

epochs_range = range(1, len(history.history['accuracy']) + 1)

# Plot 1: Training & Validation Accuracy
axes[0, 0].plot(epochs_range, history.history['accuracy'], 'b-', linewidth=2, label='Training')
axes[0, 0].plot(epochs_range, history.history['val_accuracy'], 'r-', linewidth=2, label='Validation')
axes[0, 0].set_title('Training and Validation Accuracy', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Epochs')
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].text(0.5, 0.5, 'Note: Val accuracy ≈ 0%\n(Class-disjoint validation)', 
                transform=axes[0, 0].transAxes, fontsize=10, 
                ha='center', va='center', bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.3))

# Plot 2: Training & Validation Loss
axes[0, 1].plot(epochs_range, history.history['loss'], 'b-', linewidth=2, label='Training')
axes[0, 1].plot(epochs_range, history.history['val_loss'], 'r-', linewidth=2, label='Validation')
axes[0, 1].set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Epochs')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: ROC-AUC Over Time
if 'val_roc_auc' in history.history:
    axes[1, 0].plot(epochs_range, history.history['val_roc_auc'], 'g-', linewidth=2)
    axes[1, 0].set_title('Validation ROC-AUC Over Time', fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel('Epochs')
    axes[1, 0].set_ylabel('ROC-AUC')
    axes[1, 0].grid(True, alpha=0.3)
    
    # Mark best
    best_epoch = np.argmax(history.history['val_roc_auc'])
    best_auc = history.history['val_roc_auc'][best_epoch]
    axes[1, 0].scatter(best_epoch + 1, best_auc, color='red', s=200, zorder=5)
    axes[1, 0].annotate(f'Best: {best_auc:.4f}', 
                       xy=(best_epoch + 1, best_auc),
                       xytext=(10, 10), textcoords='offset points',
                       bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.8))

# Plot 4: Learning Rate Schedule
if 'lr' in history.history:
    axes[1, 1].plot(epochs_range, history.history['lr'], 'purple', linewidth=2)
    axes[1, 1].set_title('Learning Rate Schedule', fontsize=14, fontweight='bold')
    axes[1, 1].set_xlabel('Epochs')
    axes[1, 1].set_ylabel('Learning Rate')
    axes[1, 1].set_yscale('log')
    axes[1, 1].grid(True, alpha=0.3)
else:
    axes[1, 1].axis('off')

plt.tight_layout()
plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Training history plot saved as 'training_history.png'\n")



In [ ]:
# ============================================================================
# CELL 14: Plot ROC Curve
# ============================================================================

print("Plotting ROC curve...")

plt.figure(figsize=(10, 8))
plt.plot(fpr, tpr, color='darkorange', lw=2, 
         label=f'ROC curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', 
         label='Random Classifier')

# Mark important points
for far_target in [0.001, 0.0001]:
    idx = np.argmin(np.abs(fpr - far_target))
    plt.scatter(fpr[idx], tpr[idx], s=100, zorder=5)
    plt.annotate(f'FAR={far_target:.4f}\nTAR={tpr[idx]:.4f}', 
                xy=(fpr[idx], tpr[idx]),
                xytext=(10, -10), textcoords='offset points',
                bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)', fontsize=12)
plt.ylabel('True Positive Rate (TPR)', fontsize=12)
plt.title('ROC Curve - Face Verification Performance', fontsize=14, fontweight='bold')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('roc_curve.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ ROC curve saved as 'roc_curve.png'\n")




In [ ]:
# ============================================================================
# CELL 15: Plot Precision-Recall Curve
# ============================================================================

print("Plotting Precision-Recall curve...")

plt.figure(figsize=(10, 8))
plt.plot(recall, precision, color='darkgreen', lw=2, 
         label=f'AP = {avg_precision:.4f}')
plt.fill_between(recall, precision, alpha=0.2, color='green')

# Add F1-score contours
f1_scores_plot = np.linspace(0.1, 0.9, 9)
for f1 in f1_scores_plot:
    x = np.linspace(0.01, 1)
    y = f1 * x / (2 * x - f1)
    y = np.where(y >= 0, y, np.nan)
    plt.plot(x, y, color='gray', alpha=0.2, linestyle='--', linewidth=1)
    if f1 in [0.3, 0.5, 0.7, 0.9]:
        plt.text(0.9, y[-1] - 0.03, f'F1={f1:.1f}', fontsize=8, alpha=0.5)

plt.xlabel('Recall', fontsize=12)
plt.ylabel('Precision', fontsize=12)
plt.title('Precision-Recall Curve', fontsize=14, fontweight='bold')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.legend(loc="lower left")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('precision_recall_curve.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Precision-Recall curve saved as 'precision_recall_curve.png'\n")



In [ ]:
# ============================================================================
# CELL 16: Plot Similarity Distribution
# ============================================================================

print("Plotting similarity distribution...")

# Separate positive and negative pairs
pos_scores = scores[pair_labels == 1]
neg_scores = scores[pair_labels == 0]

plt.figure(figsize=(16, 6))

# Histogram
plt.subplot(1, 2, 1)
plt.hist(pos_scores, bins=100, alpha=0.6, color='blue', 
         label=f'Intra-class (n={len(pos_scores):,})', density=True)
plt.hist(neg_scores, bins=100, alpha=0.6, color='red', 
         label=f'Inter-class (n={len(neg_scores):,})', density=True)
plt.xlabel('Cosine Similarity', fontsize=12)
plt.ylabel('Density', fontsize=12)
plt.title('Intra vs Inter-Class Similarity Distribution', fontsize=14, fontweight='bold')
plt.legend(loc='upper left')
plt.grid(True, alpha=0.3)

# Statistics - moved to a better location
pos_mean = np.mean(pos_scores)
pos_std = np.std(pos_scores)
neg_mean = np.mean(neg_scores)
neg_std = np.std(neg_scores)

# Find empty space in the plot for text placement
# Place text in upper right corner with better formatting
stats_text = (f'Intra-class (Same):\n'
              f'  Mean: {pos_mean:.4f}\n'
              f'  Std: {pos_std:.4f}\n'
              f'  Min: {np.min(pos_scores):.4f}\n'
              f'  Max: {np.max(pos_scores):.4f}\n\n'
              f'Inter-class (Different):\n'
              f'  Mean: {neg_mean:.4f}\n'
              f'  Std: {neg_std:.4f}\n'
              f'  Min: {np.min(neg_scores):.4f}\n'
              f'  Max: {np.max(neg_scores):.4f}')

plt.text(0.98, 0.98, stats_text,
         transform=plt.gca().transAxes, fontsize=9,
         verticalalignment='top', horizontalalignment='right',
         bbox=dict(boxstyle='round', facecolor='white', alpha=0.9, pad=0.5),
         family='monospace')  # Monospace font for alignment

# Box plot
plt.subplot(1, 2, 2)
box_data = [pos_scores, neg_scores]
bp = plt.boxplot(box_data, labels=['Intra-class\n(Same)', 'Inter-class\n(Different)'], 
                 patch_artist=True, widths=0.6)
bp['boxes'][0].set_facecolor('lightblue')
bp['boxes'][1].set_facecolor('lightcoral')

# Add mean markers
plt.scatter([1], [pos_mean], color='darkblue', s=100, zorder=3, marker='_', linewidth=2, label='Mean')
plt.scatter([2], [neg_mean], color='darkred', s=100, zorder=3, marker='_', linewidth=2)

plt.ylabel('Cosine Similarity', fontsize=12)
plt.title('Similarity Distribution Comparison', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, axis='y')

# Add statistics to box plot as well
plt.text(0.02, 0.98, f'Δ Mean: {pos_mean - neg_mean:.4f}',
         transform=plt.gca().transAxes, fontsize=10,
         verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))

plt.tight_layout()
plt.savefig('similarity_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Similarity distribution saved as 'similarity_distribution.png'\n")

# Additional statistics printout
print("="*60)
print("SIMILARITY STATISTICS SUMMARY:")
print("="*60)
print(f"Intra-class (Same Class Pairs):")
print(f"  Count: {len(pos_scores):,}")
print(f"  Mean ± Std: {pos_mean:.4f} ± {pos_std:.4f}")
print(f"  Range: [{np.min(pos_scores):.4f}, {np.max(pos_scores):.4f}]")
print(f"  Median: {np.median(pos_scores):.4f}")
print()
print(f"Inter-class (Different Class Pairs):")
print(f"  Count: {len(neg_scores):,}")
print(f"  Mean ± Std: {neg_mean:.4f} ± {neg_std:.4f}")
print(f"  Range: [{np.min(neg_scores):.4f}, {np.max(neg_scores):.4f}]")
print(f"  Median: {np.median(neg_scores):.4f}")
print()
print(f"Separation: {pos_mean - neg_mean:.4f} (higher is better)")
print(f"Overlap area (approx): {np.sum(pos_scores < neg_mean) / len(pos_scores) * 100:.2f}%")
print("="*60)

In [ ]:
from sklearn.metrics import roc_curve
import numpy as np
import matplotlib.pyplot as plt

def plot_tar_far_det(scores, pair_labels):
    """
    scores       : cosine similarity scores
    pair_labels  : 1 = same identity, 0 = different identity
    """
    fpr, tpr, thresholds = roc_curve(pair_labels, scores)

    plt.figure(figsize=(8, 6))
    plt.semilogx(fpr, tpr, linewidth=2)
    plt.grid(True, which="both", linestyle="--", alpha=0.6)

    plt.xlabel("False Acceptance Rate (FAR) [log scale]")
    plt.ylabel("True Acceptance Rate (TAR)")
    plt.title("TAR vs FAR Curve (DET-style)")

    # Mark standard biometric operating points
    for far in [1e-2, 1e-3, 1e-4]:
        idx = np.argmin(np.abs(fpr - far))
        plt.scatter(fpr[idx], tpr[idx])
        plt.text(
            fpr[idx], tpr[idx],
            f"FAR={far:.0e}\nTAR={tpr[idx]:.3f}",
            fontsize=9
        )

    plt.show()


In [ ]:
def plot_threshold_vs_far_tar(scores, pair_labels):
    fpr, tpr, thresholds = roc_curve(pair_labels, scores)

    plt.figure(figsize=(9, 6))
    plt.plot(thresholds, fpr, label="FAR", linewidth=2)
    plt.plot(thresholds, tpr, label="TAR", linewidth=2)

    plt.xlabel("Similarity Threshold")
    plt.ylabel("Rate")
    plt.title("Threshold vs FAR / TAR")
    plt.grid(True, linestyle="--", alpha=0.6)
    plt.legend()
    plt.show()


In [ ]:
plot_tar_far_det(scores, pair_labels)
plot_threshold_vs_far_tar(scores, pair_labels)


In [ ]:
from collections import defaultdict

def compute_intra_class_variance(embeddings, labels):
    """
    embeddings : (N, 512)
    labels     : identity labels
    """
    identity_embs = defaultdict(list)

    for emb, lbl in zip(embeddings, labels):
        identity_embs[lbl].append(emb)

    identity_variance = {}
    for lbl, embs in identity_embs.items():
        embs = np.stack(embs)
        centroid = np.mean(embs, axis=0)
        distances = np.linalg.norm(embs - centroid, axis=1)
        identity_variance[lbl] = distances

    return identity_variance

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_intra_class_variance_boxplot(identity_variance, top_k=30):
    """
    Visualizes Top-K identities with highest intra-class variance
    in a thesis-quality, readable format.
    """

    # Sort identities by mean variance (descending)
    sorted_items = sorted(
        identity_variance.items(),
        key=lambda x: np.mean(x[1]),
        reverse=True
    )[:top_k]

    labels = [str(k) for k, _ in sorted_items]
    data = [v for _, v in sorted_items]
    means = [np.mean(v) for v in data]

    fig, ax = plt.subplots(figsize=(13, 0.45 * top_k + 2))

    box = ax.boxplot(
        data,
        vert=False,
        showfliers=False,
        patch_artist=True,
        widths=0.6
    )

    # Subtle coloring (professional look)
    for patch in box["boxes"]:
        patch.set_facecolor("#dbeafe")   # light blue
        patch.set_edgecolor("#1e40af")   # dark blue
        patch.set_linewidth(1.2)

    # Median line styling
    for median in box["medians"]:
        median.set_color("#dc2626")      # red
        median.set_linewidth(1.5)

    # Mean variance overlay
    ax.scatter(
        means,
        range(1, top_k + 1),
        color="#065f46",
        marker="D",
        s=35,
        label="Mean Variance"
    )

    ax.set_yticks(range(1, top_k + 1))
    ax.set_yticklabels(labels, fontsize=9)
    ax.set_xlabel("Embedding Distance from Class Centroid", fontsize=11)
    ax.set_title(
        f"Top-{top_k} Identities with Highest Intra-Class Variance",
        fontsize=13,
        pad=12
    )

    ax.grid(axis="x", linestyle="--", alpha=0.5)
    ax.legend(loc="lower right")

    plt.tight_layout()
    plt.show()


In [ ]:
identity_variance = compute_intra_class_variance(
    embeddings=val_embeddings,
    labels=val_labels
)

plot_intra_class_variance_boxplot(identity_variance, top_k=30)


In [ ]:
# ============================================================================
# CELL 17: Summary Report
# ============================================================================

print("\n" + "="*70)
print("FINAL PERFORMANCE SUMMARY")
print("="*70)
print("\n📊 Key Metrics:")
print(f"  ROC-AUC:              {roc_auc:.4f}")
print(f"  Average Precision:    {avg_precision:.4f}")
print(f"  Best F1 Score:        {best_f1:.4f}")
print(f"  Optimal Threshold:    {best_threshold:.4f}")

print("\n🎯 Performance at Different Security Levels:")
print(f"  TAR @ FAR=0.1%:       {tar_at_far[0.001]:.4f}  (1 in 1,000)")
print(f"  TAR @ FAR=0.01%:      {tar_at_far[0.0001]:.4f}  (1 in 10,000)")

print("\n📈 Training Summary:")
print(f"  Total epochs:         {len(history.history['accuracy'])}")
if 'val_roc_auc' in history.history:
    print(f"  Best ROC-AUC:         {max(history.history['val_roc_auc']):.4f}")
    print(f"  Final ROC-AUC:        {history.history['val_roc_auc'][-1]:.4f}")

print("\n📁 Generated Files:")
print("  - best_arcface_model.keras")
print("  - embedding_model.keras")
print("  - training_history.json")
print("  - training_history.png")
print("  - roc_curve.png")
print("  - precision_recall_curve.png")
print("  - similarity_distribution.png")

print("\n💡 Interpretation:")
if roc_auc >= 0.90:
    print("  ⭐⭐⭐⭐⭐ EXCELLENT - Model shows strong generalization!")
elif roc_auc >= 0.85:
    print("  ⭐⭐⭐⭐ VERY GOOD - Model is production-ready!")
elif roc_auc >= 0.80:
    print("  ⭐⭐⭐ GOOD - Model works well, room for improvement")
else:
    print("  ⭐⭐ FAIR - Model needs improvement")

print("\n🎯 Recommended Use Cases:")
if tar_at_far[0.001] >= 0.80:
    print("  ✅ Mobile phone unlock")
    print("  ✅ Access control systems")
    print("  ✅ Photo organization")
if tar_at_far[0.0001] >= 0.70:
    print("  ✅ Payment authentication (with fallback)")
    print("  ✅ Secure facility access")

print("\n⚠️  Important Notes:")
print("  - This is class-disjoint validation (most realistic)")
print("  - Validation accuracy ≈ 0% is EXPECTED (different identities)")
print("  - ROC-AUC measures ability to distinguish same vs. different person")
print("  - Real-world performance depends on image quality and conditions")

print("\n" + "="*70)
print("✅ EVALUATION COMPLETE!")
print("="*70 + "\n")



In [ ]:
# ============================================================================
# OPTIONAL CELL 18: Save Embeddings for Future Use
# ============================================================================

print("Saving validation embeddings for future analysis...")

# Save embeddings and labels
np.save('val_embeddings.npy', val_embeddings)
np.save('val_labels.npy', val_labels)

# Save metrics
metrics = {
    'roc_auc': float(roc_auc),
    'avg_precision': float(avg_precision),
    'best_f1': float(best_f1),
    'best_threshold': float(best_threshold),
    'tar_at_far': {str(k): float(v) for k, v in tar_at_far.items()},
    'intra_class_mean': float(np.mean(pos_scores)),
    'inter_class_mean': float(np.mean(neg_scores))
}

with open('evaluation_metrics.json', 'w') as f:
    json.dump(metrics, f, indent=4)

print("✅ Embeddings and metrics saved")
print("  - val_embeddings.npy")
print("  - val_labels.npy")
print("  - evaluation_metrics.json\n")

print("="*70)
print("🎉 PIPELINE COMPLETE!")
print("="*70)
print("\nAll training, evaluation, and visualization steps completed successfully!")
print("Your model is ready for deployment or further analysis.\n")

In [ ]:
!zip -r final_year_project_face_recognition.zip \
best_arcface_model.keras \
embedding_model.keras \
evaluation_metrics.json \
roc_curve.png \
precision_recall_curve.png \
similarity_distribution.png \
training_history.png \
val_embeddings.npy \
val_labels.npy


In [ ]:
# === One-cell setup for ResNet Class-Disjoint Model artifacts ===

# 1. Create folder
!mkdir -p "ResNet_Class_Disjoint_Model"

# 2. Copy required files into the folder
!cp best_arcface_model.keras ResNet_Class_Disjoint_Model/
!cp embedding_model.keras ResNet_Class_Disjoint_Model/
!cp evaluation_metrics.json ResNet_Class_Disjoint_Model/
!cp roc_curve.png ResNet_Class_Disjoint_Model/
!cp precision_recall_curve.png ResNet_Class_Disjoint_Model/
!cp similarity_distribution.png ResNet_Class_Disjoint_Model/
!cp training_history.png ResNet_Class_Disjoint_Model/
!cp val_embeddings.npy ResNet_Class_Disjoint_Model/
!cp val_labels.npy ResNet_Class_Disjoint_Model/

# 3. Verify contents
print("📁 Folder contents:")
!ls -lh ResNet_Class_Disjoint_Model

# 4. (Optional but recommended) Zip the folder for easy download
!zip -r ResNet_Class_Disjoint_Model.zip ResNet_Class_Disjoint_Model

print("\n✅ Setup complete.")
print("➡️ Now click: Save Version → Save Output to persist for next session.")
